In [1]:
# Import Libraries
import cv2  # Importing the OpenCV library for image processing
import os  # Importing the OS library for changing the current working directory
import h5py  # Importing the h5py library for saving and loading the dataset
import pandas as pd  # Importing the pandas library for saving and loading the dataset
import numpy as np  # Importing NumPy,which is the fundamental package for scientific computing with Python
import tensorflow as tf  # Importing the TensorFlow library for deep learning model training and evaluation
import tensorflow_hub as hub  # Importing the TensorFlow Hub library for transfer learning

from PIL import Image  # Importing the Image function from PIL

# Setting GPU device to run tensorflow
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Check whether the GPU is available or not
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


In [2]:
trafficSignClasses = {}

# Defining the function to load the Traffic Sign Classes from the CSV file
data = pd.read_csv("trafficSignClasses.csv")
for i in range(len(data)):
    trafficSignClasses[data["ClassId"][i]] = data["ClassName"][i]

print(trafficSignClasses)

{0: 'Speed limit (20km/h)', 1: 'Speed limit (30km/h)', 2: 'Speed limit (50km/h)', 3: 'Speed limit (60km/h)', 4: 'Speed limit (70km/h)', 5: 'Speed limit (80km/h)', 6: 'End of speed limit (80km/h)', 7: 'Speed limit (100km/h)', 8: 'Speed limit (120km/h)', 9: 'No passing', 10: 'No passing veh over 3.5 tons', 11: 'Right-of-way at intersection', 12: 'Priority road', 13: 'Yield', 14: 'Stop', 15: 'No vehicles', 16: 'Veh > 3.5 tons prohibited', 17: 'No entry', 18: 'General caution', 19: 'Dangerous curve left', 20: 'Dangerous curve right', 21: 'Double curve', 22: 'Bumpy road', 23: 'Slippery road', 24: 'Road narrows on the right', 25: 'Road work', 26: 'Traffic signals', 27: 'Pedestrians', 28: 'Children crossing', 29: 'Bicycles crossing', 30: 'Beware of ice/snow', 31: 'Wild animals crossing', 32: 'End speed + passing limits', 33: 'Turn right ahead', 34: 'Turn left ahead', 35: 'Ahead only', 36: 'Go straight or right', 37: 'Go straight or left', 38: 'Keep right', 39: 'Keep left', 40: 'Roundabout man

In [3]:
# Defining the class for object classification
class ObjectClassification:
    # Defining the constructor for object classification
    def __init__(self):
        # Loading the trained model and save it in model

        modelUrl = "https://kaggle.com/models/google/faster-rcnn-inception-resnet-v2/frameworks/TensorFlow1/variations/faster-rcnn-openimages-v4-inception-resnet-v2/versions/1"
        # modelUrl = "https://kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow1/variations/openimages-v4-ssd-mobilenet-v2/versions/1"

        # Loading the trained model and save it in model
        self.model = hub.load(modelUrl).signatures["default"]

    # Defining the function to preprocess the image
    def preprocessImage(self, image):
        image = cv2.cvtColor(
            image, cv2.COLOR_BGR2RGB
        )  # Converting the image from BGR to RGB

        image = tf.image.convert_image_dtype(image, tf.float32)[
            tf.newaxis, ...
        ]  # Converting the image to float32 data type
        return image

    # Defining the function to detect objects in the image
    def detect(self, image):
        image = self.preprocessImage(image)  # Preprocessing the image

        # Running the model on the image
        detections = self.model(image)

        # Extracting the bounding boxes, classes, and scores from the detection
        boundingBoxes = detections["detection_boxes"].numpy()
        classLabels = detections["detection_class_labels"].numpy()
        classNames = detections["detection_class_entities"].numpy()
        scores = detections["detection_scores"].numpy()

        results = []  # Initializing the results list

        # Filtering out the detections with a confidence score less than 0.5
        for i in range(len(boundingBoxes)):
            if scores[i] > 0.25:
                results.append(
                    (
                        classNames[i].decode("utf-8"),
                        classLabels[i],
                        scores[i],
                        boundingBoxes[i],
                    )
                )

        return results

In [4]:
# Defining the class for traffic sign detection
class TrafficSignDetector:
    # Defining the constructor for traffic sign detection
    def __init__(self):
        # Loading the trained model
        self.model = tf.keras.models.load_model("model/trained_model.keras")
        self.width = 30
        self.height = 30
        self.channels = 3

    # Defining the function to preprocess the image
    def preprocessImage(self, image):
        values = []  # Initializing the values list
        image = Image.fromarray(image, "RGB")  # Converting the image to RGB format
        image = image.resize((self.width, self.height))  # Resizing the image to 30x30
        values.append(np.array(image))  # Appending the image to the image list
        finalImage = np.array(values)  # Converting the image list to a NumPy array
        finalImage = finalImage / 255  # Normalizing the image
        return finalImage

    # Defining the function to detect traffic signs in the image
    def detect(self, image):
        finalImage = self.preprocessImage(image)  # Preprocessing the image

        # Running the model on the image
        predictions = self.model.predict(finalImage, verbose=0)

        # Extracting the class label with the highest probability
        trafficLabel = np.argmax(predictions, axis=1)[0]

        return trafficSignClasses[trafficLabel]

In [5]:
# Defining the main function
if __name__ == "__main__":
    # Creating an instance of the ObjectClassification class
    objectClassification = ObjectClassification()

    # Creating an instance of the TrafficSignDetector class
    trafficSignDetector = TrafficSignDetector()

    # Create a VideoCapture object and read from the camera (0 stands for the default camera)
    cap = cv2.VideoCapture('./20231124_144733.mp4')
    # cap = cv2.VideoCapture(0)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file")

    # Output video
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter("output-live.avi", fourcc, fps, (width, height))

    # Keep reading from the camera
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        # Check if frame is not empty
        if not ret:
            break

        # Detecting objects in the frame
        results = objectClassification.detect(frame)

        # Looping over the results
        for result in results:
            # Extracting the class label, confidence score, and bounding box coordinates
            (className, classLabel, score, boundingBox) = result

            # Extracting the bounding box coordinates
            (startY, startX, endY, endX) = boundingBox

            # Convert the bounding box coordinates from relative coordinates to absolute coordinates based on the frame resolution
            startX = int(startX * frame.shape[1])
            startY = int(startY * frame.shape[0])
            endX = int(endX * frame.shape[1])
            endY = int(endY * frame.shape[0])

            # Check if the detected object is a traffic sign
            if classLabel == 96 or classLabel == 183:
                # Drawing the bounding box on the frame along with the class label and confidence score
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)

                # Region of interest (ROI) extraction
                roi = frame[startY:endY, startX:endX]

                # Detecting the traffic sign in the ROI
                trafficSign = trafficSignDetector.detect(roi)

                # Displaying the class label and confidence score
                cv2.putText(
                    frame,
                    trafficSign + ": " + str(round(score * 100, 2)) + "%",
                    (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.45,
                    (0, 0, 255),
                )

        # Writing the frame into the output video
        out.write(frame)

        # Displaying the frame
        cv2.imshow("Frame", frame)

        # Press Q on keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # When everything done, release the video capture object
    cap.release()
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
